In [1]:
from collections import defaultdict
from timeit import timeit

# Zaimplementuj algorytmy wyszukiwania wzorców

In [2]:
def naive_match(text, pattern):
    l = len(pattern)
    for i in range(len(text) - l + 1):
        if pattern == text[i:i+l]:
            yield i

In [3]:
def build_transition_table(pattern):
    alphabet, transition_table = set(pattern), []
    for q in range(len(pattern) + 1):
        state, x = defaultdict(int), min(len(pattern), q + 1)
        for a in alphabet:
            for k in range(x, -1, -1):
                if (pattern[:q] + a).endswith(pattern[:k]):
                    break
            state[a] = k
        transition_table.append(state)
    return transition_table

In [4]:
def finite_automat_match(text, tt):
    state, final_state = 0, len(tt) - 1
    offset = final_state - 1
    for i, c in enumerate(text):
        state = tt[state][c]
        if state == final_state:
            yield i - offset

In [5]:
def prepare_prefix(pattern):
    pi, k = [0], 0
    for c in pattern[1:]:
        while(k > 0 and pattern[k] != c): k = pi[k-1]
        if(pattern[k] == c): k = k + 1
        pi.append(k)
    return [0] + pi

In [6]:
def kmp_match(text, pattern, pi):
    q, l = 0, len(pattern)
    for i, c in enumerate(text):
        while(q > 0 and pattern[q] != c): q = pi[q]
        if(pattern[q] == c): q += 1
        if(q == l):
            yield i + 1 - q
            q = pi[q]

# Zaimplementuj testy porównujące szybkość działania wyżej wymienionych algorytmów.

In [7]:
def avg_time(f, times=10):
    t0 = timeit(f, number=1)
    if t0 > 2: return t0
    return (t0 + timeit(f, number=times-1)) / times

def test(text, pattern, test_name=None):
    if test_name:
        print(test_name)
    
    prepare_fin = avg_time(lambda: build_transition_table(pattern))
    prepare_kmp = avg_time(lambda: prepare_prefix(pattern))
    tt = build_transition_table(pattern)
    pf = prepare_prefix(pattern)

    naive = avg_time(lambda: list(naive_match(text, pattern)))
    print("naive\t", naive)
    fin = avg_time(lambda: list(finite_automat_match(text, tt)))
    print("finite\t", fin, "+", prepare_fin, "preparation")
    kmp = avg_time(lambda: list(kmp_match(text, pattern, pf)))
    print("kmp\t", kmp, "+", prepare_kmp, "preparation")
    print()

# Znajdź wszystkie wystąpienia wzorca "art" w załączonej ustawie, za pomocą każdego algorytmu.

In [8]:
with open("ustawa.txt") as file:
    text, pattern = file.read(), "art"
    
    tt = build_transition_table(pattern)
    pf = prepare_prefix(pattern)
    
    naive = list(naive_match(text, pattern))
    print(" naive[:10]", naive[:10])
    fin = list(finite_automat_match(text, tt))
    print("finite[:10]", fin[:10])
    kmp = list(kmp_match(text, pattern, pf))
    print("   kmp[:10]", kmp[:10])

 naive[:10] [1156, 1505, 4692, 4734, 4879, 5082, 5148, 5949, 6039, 7266]
finite[:10] [1156, 1505, 4692, 4734, 4879, 5082, 5148, 5949, 6039, 7266]
   kmp[:10] [1156, 1505, 4692, 4734, 4879, 5082, 5148, 5949, 6039, 7266]


# Porównaj szybkość działania algorytmów dla problemu z p. 3.

In [9]:
with open("ustawa.txt") as file:
    test(file.read(), "art", file.name)

ustawa.txt
naive	 0.05351316830001451
finite	 0.021237090000067838 + 2.0467399917833972e-05 preparation
kmp	 0.023262275700108147 + 1.2983999113203027e-06 preparation



# Porównaj szybkość działania algorytmów poprzez wyszukanie słowa "kruszwil" we fragmencie polskiej Wikipedii

In [10]:
with open("wikipedia-tail-kruszwil.txt") as file:
    test(file.read(), "kruszwil", file.name)

wikipedia-tail-kruszwil.txt
naive	 31.134453668999413
finite	 21.655999339000118 + 9.096399990085046e-05 preparation
kmp	 28.224473557998863 + 1.2628999684238806e-06 preparation



# Zaproponuj tekst oraz wzorzec, dla którego zmierzony czas działania algorytmów 2 oraz 3 będzie co najmniej 2-krotnie krótszy niż dla algorytmu naiwnego.

# Zaproponuj wzorzec, dla którego zmierzony czas obliczenia tablicy przejścia automatu skończonego będzie co najmniej dwukrotnie dłuższy, niż czas potrzebny na utworzenie funkcji przejścia w algorytmie KMP.

In [12]:
pattern = 'a' * 100000 # ten sam wzorzec
text = (pattern[:-1] + 'b') * 10
test(text, pattern)

naive	 3.6342687899996236
finite	 0.07437620030013932 + 0.5293011024001316 preparation
kmp	 0.17854965400001674 + 0.012966298799983633 preparation

